Import necessary libraries and save the audio data into a bytes object variable. 

In [1]:
# import wave library and read the each audio file into bytes object
# also record the number of channels, sample width and frame rate 
# into appropriate variables for the decoding process

import wave
import math
import os
from array import *

w = wave.open('Exercise2_Files/Sound1.wav', "rb")

binary_data1 = w.readframes(w.getnframes())

chan1 = w.getnchannels()
width1 = w.getsampwidth()
rate1 = w.getframerate()

print(chan1)
print(width1)
print(rate1)

w.close()

print("-----")

w = wave.open('Exercise2_Files/Sound2.wav', "rb")

binary_data2 = w.readframes(w.getnframes())

chan2 = w.getnchannels()
width2 = w.getsampwidth()
rate2 = w.getframerate()

print(chan2)
print(width2)
print(rate2)

w.close()

K = 4
divisor = int(math.pow(2, K))

1
2
44100
-----
1
2
44100


Implement the rice code algorithm.

In [2]:
# function to perform rice coding on the integers returned from
# the wave files

def rice_code(S,M):
    # calculate the value of K from the parameter M
    K = int(math.ceil(math.log(M,2)))
    # calculate the remainder of S divided by M
    r = S % M
    # calculate the quotient of S divided by M
    q = int(math.floor(S / M))
    # initialize the string of leading ones and then add as
    # many ones to the string as the quotient of S and M
    leading_ones = ""
    for i in range(q):
        leading_ones = leading_ones + "1"
    # create a string to format the binary string of the remainder
    bin_format_r = "{0:0" + str(K) + "b}"
    # get the binary string of the remainder
    binary_r = bin_format_r.format(r)
    # create the final output string
    rice_code = leading_ones + "0" + str(binary_r)
    
    return rice_code


Write the encoded strings to .ex2 file. 

In [3]:
def write_enc_to_file(data, path, M):    
    # initialize an array and store the results of passing the wave file
    # integer data to the rice coding function
    rice_arr = []
    for x in data:
        rice_arr.append(rice_code(x, M))

    print(len(rice_arr))
    # open file given by path to write encoded data 
    f = open(path, "w")
    # write data to file line by line
    for x in range(len(rice_arr)):
        temp = rice_arr[x] + "\n"
        f.write(temp)
    f.close()

In [4]:
# encode wave file data and write to new file
write_enc_to_file(binary_data1, "Sound1_Enc.ex2", divisor)
write_enc_to_file(binary_data2, "Sound2_Enc.ex2", divisor)

1002044
1008000


Convert a binary string to a decimal integer.

In [5]:
# function to convert a binary string to a decimal integer

def bin_str_to_decimal(bin_str):
    # set the remainder equal to 0 
    r = 0
    # set place value equal to 2 less than the length of the string 
    # because the strings being pass all contain the new line
    # escape sequence so they're too long by one character
    place_value = int(math.pow(2, len(bin_str) - 2))
    # iterate through the binary string except the last value 
    # which we don't care about here
    for i in range(len(bin_str) - 1):
        # increment r by the place value multiplied by the current
        # binary value
        r += int(bin_str[i]) * place_value
        # divide the place value by 2 to move on to the next place value
        place_value = int(place_value / 2)
    return r

Convert a decimal integer to a binary string. 

In [6]:
# function to convert a decimal integer to a binary string

def decimal_to_bin_str(decimal_int):
    #initialize the binary string
    bin_str = ""
    while decimal_int > 0:
        # add the result of the decimal int modulo 2 to the binary string
        bin_str = str(decimal_int % 2) + bin_str
        # divide the decimal int by two and cast result to integer
        decimal_int = int(decimal_int / 2)
    # return resulting binary string
    return bin_str

Decode the rice encoded file.

In [7]:
# function to decode the rice coded files

def dec_file(file, M):
    # open the file in read mode
    f = open(file, "r")
    # read file line by line and store data in a list
    dec_arr = f.readlines()
    # intialize the 
    decimal_arr = []
    # iterate through the encoded array
    for i in range(len(dec_arr)):
        # initialize a variable to check if we're still iterating
        # through the leading string of 1's
        check = True
        # initialize the quotient to 0
        q = 0
        # initialize the remainder to 0 and initialize an empty remainder string
#         r = 0
        remainder = ""
        for j in range(len(dec_arr[i])):
            # if current value is 1 and check is true we're still in 
            # the initial string of 1's so we increment q
            if dec_arr[i][j] == "1" and check == True:
                q += 1
            # if current value is 0 and check is true we've just reached 
            # the end of the initial string of 1's so set check to false
            elif dec_arr[i][j] == "0" and check == True:
                check = False
            # in this case check is false so we just append the rest of 
            # the string to remainder
            else:
                remainder += dec_arr[i][j]
        # send the remainder string to the binary string to decimal function
        # to convert it to a decimal int
        r = int(bin_str_to_decimal(remainder))
        # append the decoded integer value to the decimal array
        decimal_arr.append(q * M + r)
    return decimal_arr

In [8]:
decimal_arr1 = dec_file("Sound1_Enc.ex2", divisor)
decimal_arr2 = dec_file("Sound2_Enc.ex2", divisor)

Write a given bytes object to a .wav file.

In [9]:
# function to write decoded data to wave file

def write_to_wav_file(file, data, chan, width, rate):
    # convert data to byte array object
    bytes_object = bytearray(data)
    # convert byte array object to immutable bytes object
    bytes_object = bytes(bytes_object)
    
#     f = open("demofile.ex2", "w")
#     for x in range(len(bytes_object)):
#         temp = bytes_object[x]
#         f.write(temp)
#     f.close()
    
    # open wave file with the given file name
    w = wave.open(file, "wb")
    
    w.setnchannels(chan)
    w.setsampwidth(width)
    w.setframerate(rate)
    # write the frames and print the size of the bytes_object 
    # to check that it's the same size as the original data
    w.writeframesraw(bytes_object)
    print(len(bytes_object))

    w.close()

In [10]:
write_to_wav_file('Sound1_EncDec.wav', decimal_arr1, chan1, width1, rate1)
write_to_wav_file('Sound2_EncDec.wav', decimal_arr2, chan2, width2, rate2)

1002044
1008000


==================================================================================================================

==================================================================================================================

FURTHER DEVELOPMENT

==================================================================================================================

==================================================================================================================

Implementation of the improved run length rice coding algorithm. 

In [11]:
# improved rice code function using run length encoding for 
# the leading ones in the rice codes and eliminating the
# dividing zero between quotient and remainder

def run_rice_code(S,M):
    # calculate the value of K from the parameter M
    K = int(math.ceil(math.log(M,2)))
    # calculate the remainder of S divided by M
    r = S % M
    # calculate the quotient of S divided by M
    q = int(math.floor(S / M))
    # initialize the string of leading ones and then add as
    # many ones to the string as the quotient of S and M
    leading_ones = ""
    # if the quotient is greater than zero append it to bitstring 
    # and pad with zeros to make sure it's the correct length
    if q > 0:
        # append the number of leading ones in the rice code
        # written as a binary number
        leading_ones += decimal_to_bin_str(q)
        # calculate the maximum number of bits needed for quotient
        # given M = 2^K
        q_len = int(math.ceil(math.log(int(256 / M),2)))
        # if quotient isn't equal to q_len, pad quotient with required
        # number of 0's at the beginning of the string
        if len(leading_ones) < q_len:
            for i in range(q_len - len(leading_ones)):
                leading_ones = "0" + leading_ones
        # since q > 0 start bitstring with 1 to indicate we're not going
        # straight to the remainder
        leading_ones = "1" + leading_ones
    else:
        # if q == 0 start bitstring with 0 to indicate we're going
        # directly to the remainder
        leading_ones = "0"
    # create a string to format the binary string of the remainder
    bin_format_r = "{0:0" + str(K) + "b}"
    # get the binary string of the remainder
    binary_r = bin_format_r.format(r)
    # create the final output string
    rice_code = leading_ones + str(binary_r)
    
    return rice_code

Convert a bytes object to a binary string. 

In [12]:
# function to generate a single bitstring from given
# bytes object

def generate_bin_string(data, M):    
    # initialize the binary string
    bin_str = ""
    # initialize a temporary array to hold individual rice codes
    rice_arr = []
    # append each rice code string to array for each element in 
    # the bytes object
    for x in data:
        rice_arr.append(run_rice_code(x, M))
    # append each of element of the temporary array to binary string
    for x in rice_arr:
        bin_str += x
    return bin_str

Write binary data to a .bnr file.

In [13]:
# function to write the encoded binary string to a binary file

def write_to_bnr_file(data, file_name):
    # separate string into 8 bit segments
    segments = [data[x:x + 8] for x in range(0, len(data), 8)]
    input_array = array('B')
    output_array = array('B')
    # convert bits to int and add to list
    for segment in segments:
        input_array.append(int(segment, 2))
    # write each element of array to given binary file
    with open(file_name, "wb+") as f:
        input_array.tofile(f)
        f.close()
    # read each element back from the given binary file
    with open(file_name, "rb+") as f:
        output_array.fromfile(f, len(input_array))
        f.close()
    # convert back to binary string and append to a 
    # single string
    bitstring = ""
    for x in output_array:
        bitstring += "{:08b}".format(x, "08b")
    return bitstring

In [14]:
bin_str1 = generate_bin_string(binary_data1, divisor)
bitstring1 = write_to_bnr_file(bin_str1, "Sound1_Impr_Bin_Enc.bnr")
bin_str2 = generate_bin_string(binary_data2, divisor)
bitstring2 = write_to_bnr_file(bin_str2, "Sound2_Impr_Bin_Enc.bnr")

Function to decode the improved run length rice coding algorithm.

In [15]:
# function to decode the ideas for further development binary data

def dec_binary_data(data, M):
    # initialize a decimal array
    decimal_arr = []
    # calculate the maximum length of the quotient
    q_len = int(math.ceil(math.log(int(256 / M),2)))
    # boolean variable used to determine whether or not we've 
    # moved on to the next rice code
    next_num = True
    # initialize quotient string
    quotient = ""
    # initialize remainder string
    remainder = ""
    # initialize counter to determine if we've made it to the
    # end of the quotient in the code
    q_count = 0
    # initialize counter to determine if we've made it to the 
    # end of the remainder in the code
    r_count = 0
    # variable to determine whether or not a quotient exists
    check_quotient = False
    for i in range(len(data)):
        # check if we're in a new rice code and if there's a positive quotient
        if next_num == True and data[i] == "1":
            next_num = False
            check_quotient = True
        # check if we're in a new rice code and quotient is 0 
        elif next_num == True and data[i] == "0":
            next_num = False
        # if there's a quotient check whether or not we've reached
        # the end of the quotient
        elif next_num == False and check_quotient == True and q_count < q_len:
            quotient += data[i]
            q_count += 1
            if q_count == q_len:
                check_quotient = False
        # if there's no quotient or we've reached the end of it, check
        # if we've reached the end of the remainder
        elif next_num == False and check_quotient == False and r_count < ((8 - q_len) - 1):
            remainder += data[i]
            r_count += 1
        # if we've reached the end of the code, calculate the decimal integer,
        # save it to the array and reset the check variables
        else:
            remainder += data[i]
            q = int(bin_str_to_decimal(quotient + " "))
            r = int(bin_str_to_decimal(remainder + " "))
            decimal_arr.append(q * M + r)
            q_count = 0
            r_count = 0
            quotient = ""
            remainder = ""
            check_quotient = False
            next_num = True
    return decimal_arr

In [16]:
imp_bin_data_1 = dec_binary_data(bitstring1, divisor)
imp_bin_data_2 = dec_binary_data(bitstring2, divisor)

In [17]:
write_to_wav_file('Sound1_Impr_Bin_EncDec.wav', imp_bin_data_1, chan1, width1, rate1)
write_to_wav_file('Sound2_Impr_Bin_EncDec.wav', imp_bin_data_2, chan2, width2, rate2)

1002044
1008000
